<a href="https://colab.research.google.com/github/Deepak1242/Basic-langGraph-agent/blob/main/Directional_aiagent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Step - 0 Installing the dependencies


In [2]:
!pip install -qU google-generativeai==0.8.5 google-ai-generativelanguage==0.6.15 langgraph langchain langchain-google-genai openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.9/143.9 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 764.4/764.4 kB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.2/352.2 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.3/442.3 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 368.

Step- 1 Imports and secure API keys input


In [3]:
import os
import getpass
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage


Step-2 Gemini Api key iinput

In [4]:
os.environ['GOOGLE_API_KEY'] = getpass.getpass("Enter the gemini api key")

Enter the gemini api key··········


Step-3 Initialize the gemini 1.5 flash


In [5]:
llm = ChatGoogleGenerativeAI(model= "models/gemini-1.5-flash-latest",temperature = 0.2)

Step-4 Node to ask user for symptom

In [6]:
def get_symptom(state: dict) -> dict:
  symptom = input("welcome to XYZ hospital, please enter your symptom:")
  state["symptom"] = symptom
  return state


Step-5 Node to Classify the symptoms

In [7]:
def classify_symptom(state: dict) -> dict:
  prompt = (

      "You are a helpful Medical Assistant, Classify the symptoms below into one of the categories \n"
      "-General \n -Emergency \n -mental health \n"
      f"symptom : {state['symptom']} \n "
      "Respond with one word : general,emergency or Mental health \n"
      "Example  : Input : I have fever , output : General"
      )

  response = llm.invoke([HumanMessage(content = prompt)])
  category = response.content.strip()
  print(f"LLM classifies the symptom as : {category}")
  state["category"] = category
  return state

##Step-6 Router Logic to route to the correct node

In [8]:
def symptom_router(state:dict)->dict:
  cat = state["category"].lower()
  if "general" in cat:
    return "general"
  elif "emergency" in cat:
    return "emergency"
  elif "mental health" in cat:
    return "mental health"
  else:
    return "general"

###step-7 Category-specific reponses nodes

In [9]:
def general_node(state:dict)->dict:
  state["answer"] = f" '{state['symptom']}' : seems general : directing you to general ward for consulting a doctor.."
  return state

def emergency_node(state:dict) -> dict:
  state["answer"] = f" '{state['symptom']}' : seems emergency : directing you to emergency ward for consulting a doctor.."
  return state

def mental_node(state:dict) -> dict:
  state["answer"] = f" '{state['symptom']}' : seems Mental issue : directing you to mental health specialist doctor.."
  return state

###Build a LangGraph

In [10]:
builder = StateGraph(dict)

builder.set_entry_point("get_symptom")
builder.add_node("get_symptom",get_symptom)
builder.add_node("classify",classify_symptom)
builder.add_node("general",general_node)
builder.add_node("emergency",emergency_node)
builder.add_node("mental health",mental_node)


builder.add_edge("get_symptom" , "classify")
builder.add_conditional_edges("classify",symptom_router,{
    "general": "general",
    "emergency": "emergency",
    "mental health": "mental health"
})

builder.add_edge("general",END)
builder.add_edge("emergency",END)
builder.add_edge("mental health",END)



##Step 9 Compile and invoke the graph


In [11]:
graph = builder.compile()
final_state = graph.invoke({})
print(final_state["answer"])


welcome to XYZ hospital, please enter your symptom:FEELING LITTLE WEIRD AND TIRED 
LLM classifies the symptom as : General
 'FEELING LITTLE WEIRD AND TIRED ' : seems general : directing you to general ward for consulting a doctor..
